In [12]:
# imports used to manipulate data and describe the data with plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

# NEED TO DO

I need to add the game number of that season as a column in order to take of the first n games

# POST MVP

MVP
- created two data sets
    - game by game statistics (each row accounts for a single game)
    - aggregated statistics (each row accounts for all games up to that game)
- feature selection process that produced similar but slightly worse predictions
- xgboost model predicting with 58% accuracy

Post MVP plans
- make predicitons on a single season
- identify optimal number of games for statistics to aggregate
- neural net for predictions
- allow game by game predictions usable with gui or interface of some sort

# Single Season Predictions
For this experiment I am going to be using the best performing model (XGBoost) with optimal parameters to make predictions on a single season. The test set is going to be the 2019 season.

In [13]:
# importing aggregate data and dropping unnamed
df = pd.read_csv('data/aggregate_data.csv')

In [14]:
df

,Unnamed: 0,Date,game_in_series,away_team,home_team,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_catch_interference,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_pitch_balks,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_win_loss,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_catch_interference,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_pitch_balks,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_win_loss,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,home_outcome,away_outcome
0,0,2000-03-30,0,NYN,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1,2000-04-03,0,COL,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,2,2000-04-03,0,MIL,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
3,3,2000-04-03,0,SFN,MIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,4,2000-04-03,0,LAN,WAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48563,48563,2019-09-29,0,DET,CHA,4.393750,34.381250,8.99375,1.612500,0.125000,1.131250,4.200000,0.225000,0.200000,0.393750,2.356250,0.081250,9.606250,0.393750,0.175000,0.706250,0.006250,6.675000,4.318750,4.793750,4.787500,0.443750,0.031250,26.318750,9.437500,0.731250,0.081250,1.050000,0.00625,0.443750,0.308682,0.256513,6.125000,0.406390,0.324657,0.149877,4.136388,14.250000,4.699443,0.315905,3.618750,34.487500,8.287500,1.818750,0.256250,0.918750,3.456250,0.056250,0.262500,0.300000,2.425000,0.087500,9.918750,0.350000,0.125000,0.662500,0.000000,6.656250,4.587500,5.206250,5.193750,0.406250,0.043750,26.718750,9.031250,0.681250,0.100000,0.787500,0.000000,0.293750,0.286314,0.234660,5.293750,0.379340,0.300

In [15]:
# convering date column from object to datetime
df.Date = pd.to_datetime(df.Date,format='%Y-%m-%d')

In [16]:
df

,Unnamed: 0,Date,game_in_series,away_team,home_team,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_catch_interference,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_pitch_balks,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_win_loss,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_catch_interference,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_pitch_balks,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_win_loss,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,home_outcome,away_outcome
0,0,2000-03-30,0,NYN,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1,2000-04-03,0,COL,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,2,2000-04-03,0,MIL,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
3,3,2000-04-03,0,SFN,MIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,4,2000-04-03,0,LAN,WAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48563,48563,2019-09-29,0,DET,CHA,4.393750,34.381250,8.99375,1.612500,0.125000,1.131250,4.200000,0.225000,0.200000,0.393750,2.356250,0.081250,9.606250,0.393750,0.175000,0.706250,0.006250,6.675000,4.318750,4.793750,4.787500,0.443750,0.031250,26.318750,9.437500,0.731250,0.081250,1.050000,0.00625,0.443750,0.308682,0.256513,6.125000,0.406390,0.324657,0.149877,4.136388,14.250000,4.699443,0.315905,3.618750,34.487500,8.287500,1.818750,0.256250,0.918750,3.456250,0.056250,0.262500,0.300000,2.425000,0.087500,9.918750,0.350000,0.125000,0.662500,0.000000,6.656250,4.587500,5.206250,5.193750,0.406250,0.043750,26.718750,9.031250,0.681250,0.100000,0.787500,0.000000,0.293750,0.286314,0.234660,5.293750,0.379340,0.300

In [18]:
len(df[(df.Date.dt.year==2019)&(df.home_team=='NYA')].head(500))

81

In [21]:
df

,Unnamed: 0,Date,game_in_series,away_team,home_team,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_catch_interference,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_pitch_balks,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_win_loss,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_catch_interference,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_pitch_balks,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_win_loss,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,home_outcome,away_outcome
0,0,2000-03-30,0,NYN,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1,2000-04-03,0,COL,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,2,2000-04-03,0,MIL,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
3,3,2000-04-03,0,SFN,MIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,4,2000-04-03,0,LAN,WAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48563,48563,2019-09-29,0,DET,CHA,4.393750,34.381250,8.99375,1.612500,0.125000,1.131250,4.200000,0.225000,0.200000,0.393750,2.356250,0.081250,9.606250,0.393750,0.175000,0.706250,0.006250,6.675000,4.318750,4.793750,4.787500,0.443750,0.031250,26.318750,9.437500,0.731250,0.081250,1.050000,0.00625,0.443750,0.308682,0.256513,6.125000,0.406390,0.324657,0.149877,4.136388,14.250000,4.699443,0.315905,3.618750,34.487500,8.287500,1.818750,0.256250,0.918750,3.456250,0.056250,0.262500,0.300000,2.425000,0.087500,9.918750,0.350000,0.125000,0.662500,0.000000,6.656250,4.587500,5.206250,5.193750,0.406250,0.043750,26.718750,9.031250,0.681250,0.100000,0.787500,0.000000,0.293750,0.286314,0.234660,5.293750,0.379340,0.300

In [20]:
df.drop(columns=['game_number_of_season','Unnamed: 0'])

KeyError: "['game_number_of_season'] not found in axis"

In [19]:
# creating train test split manually
train_set = df[(df.Date.dt.year != 2019)]
test_set = df[(df.Date.dt.year == 2019)]

# going to order test set because if used in real time would only have games coming in order of date
test_set.sort_values(by=['Date'],inplace=True)

# going to drop the first 10 games of each season from train
train_set = train_set[(df.game_number_of_season > 10)]

AttributeError: 'DataFrame' object has no attribute 'game_number_of_season'

In [ ]:
# creating x and y from train (all games not in 2019)
X_train = train_set.drop(columns='home_outcome_nonagg')
y_train = train_set['home_outcome_nonagg']

# creating x and y from test (all 2019 games)
X_test = test_set.drop(columns='home_outcome_nonagg')
y_test = test_set['home_outcome_nonagg']

In [ ]:
xgb_model = xgb.XGBClassifier(objective = 'binary:logistic',
                                 learning_rate = 0.01,
                                 max_depth = 5,
                                 min_child_weight = 2,
                                 n_estimators = 300)
# dropping 3 columns that are not used for prediction
xgb_model.fit(X_train.drop(columns=['home_team','away_team','Date']),
              y_train)

In [ ]:
import joblib
joblib.dump(xgb_model,"xgb_final.pkl")

In [ ]:
xgb_model_predictions_train = xgb_model.predict(X_train.drop(columns=['home_team','away_team','Date']))
xgb_model_predictions_test = xgb_model.predict(X_test.drop(columns=['home_team','away_team','Date']))
train_acc = accuracy_score(y_train, xgb_model_predictions_train)
test_acc = accuracy_score(y_test, xgb_model_predictions_test)
print("Train Accuracy: %f" % (train_acc))
print("Test Accuracy: %f" % (test_acc))

In [ ]:
# looking at when we are predicting the best
# accuracy over whole season
accuracy_whole = 0
# past games right or wrongs
game_preds = []
# list to hold game rolling accuracy throuhgout whole season to see how we do as season goes on
accuracy_rolling_season = []
# number of games seen
num_of_games = 0
# correct predictions
num_right = 0
# wrong predictions
num_wrong = 0
# number of games for rolling
rolling_number = 50

for true,pred in zip(y_test,xgb_model_predictions_test):
    # after seeing rolling number of games
    if num_of_games > rolling_number:
        # get rid of least recent game, using append so most recent game at end
        game_preds = game_preds[1:]
    # every period of rolling games calculate the rolling accuracy
    if num_of_games % rolling_number == 0:
        # calcuate rolling game accuracy and append to season list
        accuracy_rolling_season.append(sum(game_preds)/rolling_number)
    # one more game seen
    num_of_games +=1
    # correct prediction
    if pred == true:
        num_right += 1
        game_preds.append(1)
    # wrong prediction
    else:
        num_wrong +=1
        game_preds.append(0)
accuracy_whole = num_right/num_of_games

print('Number of games right: {}'.format(num_right))
print('Number of games wrong: {}'.format(num_wrong))
print('Best rolling game streak: {}'.format(max(accuracy_rolling_season)))
print('Season Accuracy: {}'.format(accuracy_whole))

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.plot(accuracy_rolling_season,color='red')
ax.set_xlabel('Sets of 50 Games Seen')
ax.set_ylabel('Accuracy Over 50 Games')
ax.set_yticks(np.linspace(0,1,11))
ax.set_xticks(range(0,round(num_of_games/50),5))
ax.set_title('Accuracy over Season (50 Game Splits)')
ax.grid();

# Optimal Game Number Buffer
When using aggregated statistics it is useful to not predict on the games at the start of the season as the aggregated statistics have not had enought time to really show how well that team performs. Allowing a 10 game buffer would allow us to build up the statistics and then by the 11th game we have a much better idea on how each team will perform allowing for better predictions.

In [ ]:
def xg_prediction(buffer):
    # creating train test split manually
    X = df.drop(columns='home_outcome_nonagg')
    y = df.home_outcome_nonagg
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=49)
    # get rid of games in train before buffer number
    # combine x and y so they dont get mixed
    train = pd.concat([X_train,y_train],axis=1)
    # get rid of games before buffer game number
    train = train[(train.game_number_of_season>buffer)]
    # make x and y train
    X_train = train.drop(columns='home_outcome_nonagg')
    y_train = train.home_outcome_nonagg
    
    
    #run model
    xgb_model_buffer = xgb.XGBClassifier(objective = 'binary:logistic',
                                 learning_rate = 0.01,
                                 max_depth = 5,
                                 min_child_weight = 2,
                                 n_estimators = 300)
    # dropping 3 columns that are not used for prediction
    xgb_model_buffer.fit(X_train.drop(columns=['home_team','away_team','Date']),
                  y_train)
    
    xgb_model_buffer_predictions_train = xgb_model_buffer.predict(X_train.drop(columns=['home_team','away_team','Date']))
    xgb_model_buffer_predictions_test = xgb_model_buffer.predict(X_test.drop(columns=['home_team','away_team','Date']))
    train_acc_buffer = accuracy_score(y_train, xgb_model_buffer_predictions_train)
    test_acc_buffer = accuracy_score(y_test, xgb_model_buffer_predictions_test)
    print("Train Accuracy: %f" % (train_acc_buffer))
    print("Test Accuracy: %f" % (test_acc_buffer))
    return train_acc,test_acc

In [ ]:
scores_per_buffer = []
for buffer in range(0,35,5):
    train_acc,test_acc = xg_prediction(buffer)
    scores_per_buffer.append((train_acc,test_acc))

From this we can see that as we reduce the number of games seen when fitting the model the better we perfom on the training set but the worse we perform on the test set. This means that in the training set we should keep the entire dataset for training.

# Optimal Game Number to start predicting

In [ ]:
# creating train test split manually
X = df.drop(columns='home_outcome_nonagg')
y = df.home_outcome_nonagg

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=49)

#run model
xgb_model_buffer = xgb.XGBClassifier(objective = 'binary:logistic',
                             learning_rate = 0.01,
                             max_depth = 5,
                             min_child_weight = 2,
                             n_estimators = 300)
# dropping 3 columns that are not used for prediction
xgb_model_buffer.fit(X_train.drop(columns=['home_team','away_team','Date']),
              y_train)

xgb_model_buffer_predictions_train = xgb_model_buffer.predict(X_train.drop(columns=['home_team','away_team','Date']))
xgb_model_buffer_predictions_test = xgb_model_buffer.predict(X_test.drop(columns=['home_team','away_team','Date']))
train_acc_buffer = accuracy_score(y_train, xgb_model_buffer_predictions_train)
test_acc_buffer = accuracy_score(y_test, xgb_model_buffer_predictions_test)
print("Train Accuracy: %f" % (train_acc_buffer))
print("Test Accuracy: %f" % (test_acc_buffer))

In [ ]:

for buffer in range(0,85,5):
    test = pd.concat([X_test,y_test],axis=1)
    # get rid of games before buffer game number
    test = test[(test.game_number_of_season>buffer)]
    print(test.shape)
    # make x and y train
    X_test = test.drop(columns='home_outcome_nonagg')
    y_test = test.home_outcome_nonagg
    xgb_model_buffer_predictions_test = xgb_model_buffer.predict(X_test.drop(columns=['home_team','away_team','Date']))
    test_acc_buffer = accuracy_score(y_test, xgb_model_buffer_predictions_test)
    print("Test Accuracy: {} with buffer {}".format(test_acc_buffer,buffer))

From this we can see that as time goes on we greatly improve accuracy on the test set. This means that as the season goes on we have a much better chance of predicting the outcome correctly.

# Neural Net for Predictions

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.optimizers import Adam, SGD


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models


In [ ]:
X = df.drop(columns='home_outcome_nonagg')
y = df.home_outcome_nonagg

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=49)

In [ ]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)


In [ ]:
X_train = X_train.drop(columns=['Date','home_team','away_team'])
X_test = X_test.drop(columns=['Date','home_team','away_team'])

In [ ]:
## Vanilla
# model structure
model = Sequential()

# first hidden layer
# 12288 = 64x64x3 = size of image
model.add(Dense(50,input_dim=(77),activation='relu'))

# second hidden layer
model.add(Dense(50,activation='relu'))


# output layer
model.add(Dense(2, activation='sigmoid'))

# view the structure of the neural net
#model.summary()

# creating the NN, binary_cross for classification
model.compile(loss='binary_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])

In [ ]:
# fitting the model to the train data
history = model.fit(X_train,
                    y_train,
                   epochs=8,
                   verbose=1)
# how did model perform
score = model.evaluate(X_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_new = tf.keras.Sequential([
    layers.Dense(64,input_dim=(77), activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='sigmoid')
    ])

In [ ]:
model_new.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model_new.fit(X_train, y_train,
                            epochs=8)

score = model_new.evaluate(X_test,y_test,verbose=0)
print('Test accuracy:', score[1])